This was a personal project where I tried to make an optimal schedule for my family to watch my grandmother after her stroke. ScheduleSurveyData.csv is the results of a Google Poll asking relatives for their preferred, acceptable, and impossible timeslots for the morning, afternoon, and evening each day of the week. This is imported, parsed, input into a Mixed Integer Program, then solved resulting in the schedule seen at the bottom. Names have been changed for confidentiality, and code has been updated to work with newer versions of JuMP.

In [2]:
using JuMP
using Cbc
using CSV
using DataFrames

#Parameters
default_val=1
prefer_val=.5
dislike_val=3
impossible_val=100


in_file=DataFrame(CSV.File("ScheduleSurveyData.csv",delim=','))

Row,Timestamp,What is your name?,Select any time slots which you PREFER [9AM-2PM],Select any time slots which you PREFER [2PM-6PM],Select any time slots which you PREFER [6PM-10PM],Select any time slots which you DISLIKE [9AM-2PM],Select any time slots which you DISLIKE [2PM-6PM],Select any time slots which you DISLIKE [6PM-10PM],Select any time slots which you CANNOT make [9AM-2PM],Select any time slots which you CANNOT make [2PM-6PM],Select any time slots which you CANNOT make [6PM-10PM],Write any specific requests or comments here.
,String31,String7,String?,String?,String?,String?,String?,String31?,String?,String?,String?,String?
1,2018/06/14 12:24:09 AM AST,Alice,missing,missing,Mon (6/18);Tue (6/19);Wed (6/20);Thu (6/21),Mon (6/18);Tue (6/19);Wed (6/20);Thu (6/21);Fri (6/22);Sat (6/23);Sun (6/24),missing,Thu (6/21),missing,missing,Fri (6/22);Sun (6/24),missing
2,2018/06/14 2:47:04 PM AST,Bob,missing,Sat (6/23);Sun (6/24),Mon (6/18);Tue (6/19);Wed (6/20);Thu (6/21);Fri (6/22);Sat (6/23);Sun (6/24),missing,missing,missing,Mon (6/18);Tue (6/19);Wed (6/20);Thu (6/21);Fri (6/22),Mon (6/18);Tue (6/19);Wed (6/20);Thu (6/21);Fri (6/22),missing,missing
3,2018/06/14 3:53:31 PM AST,Charlie,Mon (6/18);Wed (6/20);Thu (6/21);Fri (6/22),Mon (6/18);Tue (6/19);Wed (6/20);Thu (6/21),Sun (6/24),missing,missing,Thu (6/21);Fri (6/22),missing,missing,missing,Lynn
4,2018/06/14 6:28:11 PM AST,David,Mon (6/18);Tue (6/19);Wed (6/20);Thu (6/21);Fri (6/22);Sat (6/23);Sun (6/24),missing,Mon (6/18);Tue (6/19);Wed (6/20);Sat (6/23),missing,missing,Thu (6/21);Fri (6/22),missing,Mon (6/18);Tue (6/19);Wed (6/20);Thu (6/21);Fri (6/22);Sat (6/23);Sun (6/24),Sun (6/24),missing
5,2018/06/15 7:46:25 PM AST,Edna,missing,Sun (6/24),missing,missing,missing,missing,Mon (6/18);Tue (6/19);Wed (6/20);Thu (6/21);Fri (6/22),Mon (6/18);Tue (6/19);Wed (6/20);Thu (6/21);Fri (6/22),missing,missing
6,2018/06/15 9:30:21 PM AST,Faith,missing,missing,missing,missing,missing,missing,missing,missing,missing,I'm in a new job and don't know what my schedule will be. It's retail so it could be all over the place. Please retain me on the sub list for the time being. Thanks!
7,2018/06/17 12:45:15 AM AST,Gerald,missing,missing,Wed (6/20);Fri (6/22);Sat (6/23);Sun (6/24),Mon (6/18);Wed (6/20);Fri (6/22),Mon (6/18);Tue (6/19);Wed (6/20);Thu (6/21);Fri (6/22),missing,Fri (6/22);Sat (6/23);Sun (6/24),Fri (6/22);Sat (6/23);Sun (6/24),missing,missing
8,2018/06/21 4:26:53 PM AST,Harry,missing,missing,missing,missing,Wed (6/20),Wed (6/20),Mon (6/18);Tue (6/19);Fri (6/22);Wed (6/20),Mon (6/18);Tue (6/19);Fri (6/22),Sun (6/24);Mon (6/18);Sat (6/23);Fri (6/22),missing
9,6/15/2018 21:16,Ingrid,missing,missing,missing,missing,missing,missing,missing,missing,"Mon (6/18), Tue (6/19), Wed (6/20), Thu (6/21), Fri (6/22)",missing


In [3]:
# Some cleaning
num_people=size(in_file,1)
prefs=ones(num_people,3,7)*default_val
for j in 1:num_people
    temp_array=ones(3,7)*default_val
    # Do Prefer
    for i in 3:5
        if !ismissing(in_file[j,i])
            if contains(in_file[j,i],"Mon")
                temp_array[i-2,1]=prefer_val
            end
            if contains(in_file[j,i],"Tue")
                temp_array[i-2,2]=prefer_val
            end
            if contains(in_file[j,i],"Wed")
                temp_array[i-2,3]=prefer_val
            end
            if contains(in_file[j,i],"Thu")
                temp_array[i-2,4]=prefer_val
            end
            if contains(in_file[j,i],"Fri")
                temp_array[i-2,5]=prefer_val
            end
            if contains(in_file[j,i],"Sat")
                temp_array[i-2,6]=prefer_val
            end
            if contains(in_file[j,i],"Sun")
                temp_array[i-2,7]=prefer_val
            end
        end
    end
    # Do Dislike
    for i in 6:8
        if !ismissing(in_file[j,i])
            if contains(in_file[j,i],"Mon")
                temp_array[i-5,1]=dislike_val
            end
            if contains(in_file[j,i],"Tue")
                temp_array[i-5,2]=dislike_val
            end
            if contains(in_file[j,i],"Wed")
                temp_array[i-5,3]=dislike_val
            end
            if contains(in_file[j,i],"Thu")
                temp_array[i-5,4]=dislike_val
            end
            if contains(in_file[j,i],"Fri")
                temp_array[i-5,5]=dislike_val
            end
            if contains(in_file[j,i],"Sat")
                temp_array[i-5,6]=dislike_val
            end
            if contains(in_file[j,i],"Sun")
                temp_array[i-5,7]=dislike_val
            end
        end
    end
    # Do Impossible
    for i in 9:11
        if !ismissing(in_file[j,i])
            if contains(in_file[j,i],"Mon")
                temp_array[i-8,1]=impossible_val
            end
            if contains(in_file[j,i],"Tue")
                temp_array[i-8,2]=impossible_val
            end
            if contains(in_file[j,i],"Wed")
                temp_array[i-8,3]=impossible_val
            end
            if contains(in_file[j,i],"Thu")
                temp_array[i-8,4]=impossible_val
            end
            if contains(in_file[j,i],"Fri")
                temp_array[i-8,5]=impossible_val
            end
            if contains(in_file[j,i],"Sat")
                temp_array[i-8,6]=impossible_val
            end
            if contains(in_file[j,i],"Sun")
                temp_array[i-8,7]=impossible_val
            end
        end
    end
    prefs[j,:,:]=temp_array
end

Define the scheduling model. Scheduling an impossible slot to a person gives a large penalty to the objective, a disliked slot a small penalty, and a preferred slort a bonus. Assigning slots that are none of these give no penalty or benefit.

In [4]:
# Define the model
m = Model(Cbc.Optimizer)
@variable(m, x[1:num_people,1:3,1:7], Bin)
# No two people assigned to same time
# All slot filled
for j in 1:3
    for k in 1:7
        @constraint(m, sum(x[i,j,k] for i=1:num_people)==1)
        for i in 1:num_people
            if abs(prefs[i,j,k]-impossible_val)<1e-1
                @constraint(m, x[i,j,k]==0)
            end
        end
    end
end

# Minimum number constraint
for i in 1:num_people
    @constraint(m, sum(x[i,j,k] for j=1:3 for k=1:7) >= floor(21/num_people))
end
                
# Each person max one per day
for i in 1:num_people
    for k in 1:7
        @constraint(m, sum(x[i,j,k] for j=1:3) <= 1)
    end
end

@objective(m, Min, sum( sum(prefs[i,j,k]*x[i,j,k] for j=1:3 for k=1:7)  for i=1:num_people))
optimize!(m)

Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 15 - 0.00 seconds
Cgl0002I 62 variables fixed
Cgl0004I processed model has 79 rows, 148 columns (148 integer (148 of which binary)) and 424 elements
Cutoff increment increased from 1e-05 to 0.4999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 15
Cbc0038I Before mini branch and bound, 148 integers at bound fixed and 0 continuous
Cbc0038I Mini branch and bound did not improve solution (0.00 seconds)
Cbc0038I After 0.00 seconds - Feasibility pump exiting with objective of 15 - took 0.00 seconds
Cbc0012I Integer solution of 15 found by feasibility pump after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 15, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from

Print the optimized schedule

In [5]:
print("\tMon\tTues\tWed\tThu\tFri\tSat\tSun\n")
sol=value.(x)
curr_line="9AM-2PM\t"
for k in 1:7
    for i in 1:num_people
        if abs(sol[i,1,k]-1)<1e-5
            if ismissing(in_file[i,2])
                curr_line*=i
            else
                curr_line*=in_file[i,2][1:min(length((in_file[i,2])),7)]*"\t"
            end
        end
    end
end
print(curr_line*"\n")
curr_line="2PM-6PM\t"
for k in 1:7
    for i in 1:num_people
        if abs(sol[i,2,k]-1)<1e-5
            if ismissing(in_file[i,2])
                curr_line*=i
            else
                curr_line*=in_file[i,2][1:min(length((in_file[i,2])),7)]*"\t"
            end
        end
    end
end
print(curr_line*"\n")
curr_line="6PM-10P\t"
for k in 1:7
    for i in 1:num_people
        if abs(sol[i,3,k]-1)<1e-5
            if ismissing(in_file[i,2])
                curr_line*=i
            else
                curr_line*=in_file[i,2][1:min(length((in_file[i,2])),7)]*"\t"
            end
        end
    end
end
print(curr_line)

	Mon	Tues	Wed	Thu	Fri	Sat	Sun
9AM-2PM	Charlie	David	Faith	Harry	David	John	Harry	
2PM-6PM	Ingrid	Charlie	Ingrid	Charlie	Faith	Edna	Edna	
6PM-10P	Bob	Alice	Alice	Bob	John	Gerald	Gerald	